# Artificial Neural Network Model

### IF3170 - Machine Learning

Developed by: 
1. Juan Christopher Santoso (13521116)
2. Nicholas Liem (13521135)
3. Nathania Calista Djunaedi (13521139)
4. Antonio Natthan Krishna (13521162)

#### **Import Library**

In [137]:
import numpy as np
import pandas as pd
import json
import copy
import pickle

#### **Neural Network Properties**

#### 1. Activation Function

In [138]:
class ActivationFunction:
    def __init__(self, types='Sigmoid'):
        self.func = self.sigmoid
        self.dfunc = self.dsigmoid
        self.dfuncerr = self.dsum_square

        match types:
            case 'sigmoid':
                self.func = self.sigmoid
                self.dfunc = self.dsigmoid
                self.dfuncerr = self.dsum_square
            case 'linear':
                self.func = self.linear
                self.dfunc = self.dlinear
                self.dfuncerr = self.dsum_square
            case 'softmax':
                self.func = self.softmax
                # self.dfuncerr = self.derr_softmax
                self.dfuncerr = self.dsum_square
            case 'relu':
                self.func = self.relu
                self.dfunc = self.drelu
                self.dfuncerr = self.dsum_square

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def dsigmoid(self, x):
        sig = self.sigmoid(x)
        return sig * (1-sig)
    
    def linear(self, x):
        return x
    
    def dlinear(self, x):
        return 1
    
    def softmax(self, x):
        expX = np.exp(x - np.max(x, axis=1, keepdims=True))
        return expX / np.sum(expX, axis=1, keepdims=True)
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def drelu(self, x):
        return np.where(x > 0, 1, 0)
    
    def dsum_square(self, output, target):
        return target - output
    
    def derr_softmax(self, output, target):
        if (target != 1):
            return output
        else:
            return output - 1

#### 2. Loss Function

In [139]:
class LossFunction:
    @staticmethod
    def calculate(output, target, layers):
        activation_mode = layers[-1]['activation_function']
        if (activation_mode == "softmax"):
            return LossFunction.loss_softmax(output, target)
        else:
            return LossFunction.loss_rsl(output, target)

    @staticmethod
    def loss_rsl(output, target):
        err = 0
        for i in range (len(output)):
            err += (target[i] - output[i])**2
        return 0.5 * err
    
    @staticmethod
    def loss_softmax(output, target):
        idx = np.where(target == 1)[0]
        return -1 * np.log10(output[idx])

#### 3. Forward Propagation (Using Fast Forward Neural Network)

In [140]:
class ForwardPropagation:
    @staticmethod
    def process(input_data, layers, weights):
        activations = input_data
        neuron_net = []
        neuron_out = []

        for i in range(len(layers)):
            activations_with_bias = np.insert(activations, 0, 1, axis=1)
            net_input = np.dot(activations_with_bias, weights[i])
            activation_mode = layers[i]['activation_function']
            activationFunc = ActivationFunction(activation_mode)
            activations = activationFunc.func(net_input)
            neuron_net.append(net_input)
            neuron_out.append(activations)

        return activations, neuron_net, neuron_out

#### 4. Backward Propagation

In [141]:
class BackwardPropagation:
    @staticmethod
    def process(weights, output, target, neuron_out, layers, learning_rate, input_data):
        
        # Initiate variables
        delta = []
        delta_layer = []

        # Delta Output Layer
        activation_mode = layers[-1]['activation_function']
        activationFunc = ActivationFunction(activation_mode)

        # Handle Case for Softmax
        if (activation_mode == 'softmax'):
            print(output)
            print(target)
            for i in range (len(output)): 
                delta_layer.append(activationFunc.dfuncerr(output[i], target[i]))
            print("RES", delta_layer)
        else:
            for i in range (len(output)): 
                delta_layer.append(activationFunc.dfuncerr(output[i], target[i]) * activationFunc.dfunc(output[i]))
        delta.append(delta_layer)

        # Delta Hidden Layer
        for i in range(len(layers) - 2, -1, -1):
            activation_mode = layers[i]['activation_function']
            activationFunc = ActivationFunction(activation_mode)

            # Save previous delta layer
            # If 2nd last, then return the Output layer
            # Else, return the previous layer but exclude the Bias val
            prev_delta_layer = delta_layer if (i == (len(layers)-2)) else delta_layer[1:]

            # Initiate new Delta Layer
            delta_layer = []
            layer_weight = weights[i+1]
            layer_output = neuron_out[i][0]

            # Iterate each neuron
            for j in range (len(layer_weight)): 
                neuron_weight = layer_weight[j]
                sigma = np.dot(neuron_weight, prev_delta_layer)

                # Make sure hidden layers are not softmax
                assert activation_mode != "softmax", "Softmax cannot be in hidden layers"
                delta_layer.append(sigma * activationFunc.dfunc(1 if j == 0 else layer_output[j-1]))
            # Append but push from front
            delta = [delta_layer[1:]] + delta


        # print("LAYERS", layers)
        # print("DELTA", delta)
        # print("INPUT", input_data)
        # print("NEURON OUT", neuron_out)
        # print("WEIGHT", weights)
        # print("\n")


        # Update Weight
        # Iterate all the layers
        for i in range (len(layers)):
            if (i == 0):
                layer_input = input_data
            else :
                layer_input = neuron_out[i-1][0] # [0] because it is a list inside a list

            layer_input = np.insert(layer_input, 0, 1)

            # print("WEIGHT", weights)
            # print("Delta", delta)
            # print("LAYER INPUT", layer_input)
            
            # Iterate all weights
            for j in range (len(weights[i])):
                # Iterate all weights
                for k in range (len(weights[i][j])):
                    # It is based on formula wji = wji + learn_rate * dE/dnet * dnet/dw
                    weights[i][j][k] += learning_rate * (delta[i][k] * layer_input[j])
        
        return weights

#### **Artificial Neural Network**

In [142]:
class ArtificialNeuralNetwork:
    def __init__(self, architecture):
        self.layers = architecture.layers
        self.learning_rate = architecture.learning_rate
        self.error_threshold = architecture.error_threshold
        self.max_iter = architecture.max_iteration
        self.batch_size = architecture.batch_size
        self.input_data = architecture.input
        self.target = architecture.target
        self.weights = architecture.initial_weights
        self.expected_stopped_by = architecture.stopped_by
        self.expect_weights = architecture.final_weights

    def train(self):
        temp_weight = copy.deepcopy(self.weights)

        for j in range (self.max_iter):
            error_total = 0
            minibatch = self.batch_size
            for i in range (len(self.input_data)):
                if (minibatch == 0):
                    self.weights = copy.deepcopy(temp_weight)
                    minibatch = self.batch_size
                output, _, neuron_out = ForwardPropagation.process([self.input_data[i]], self.layers, self.weights)
                error_total += LossFunction.calculate(output[0], self.target[i], self.layers)
                temp_weight = BackwardPropagation.process(temp_weight, output[0], self.target[i], neuron_out, self.layers, self.learning_rate, self.input_data[i])
                minibatch = minibatch - 1
            
            self.weights = copy.deepcopy(temp_weight)
            if (error_total < self.error_threshold):
                break

        if (j == self.max_iter-1):
            self.stopped_by = "MAX ITERATION"
        elif (error_total < self.error_threshold):
            self.stopped_by = "ERROR THRESHOLD"
        else:
            self.stopped_by = "UNIDENTIFIED"

        print("\n")
        print("J :", j)
        print("TOTAL ERROR VAL:", error_total)
        print("\n")
        
        print("EXCPECTED")
        print("STOPPED BY :", self.expected_stopped_by)
        for weight_group in self.expect_weights:
            print("[")
            for weight in weight_group:
                print("\t", weight)
            print("]")
        print("\n")

        print("RESULT")
        print("STOPPED BY :", self.stopped_by)
        for weight_group in self.weights:
            print("[")
            for weight in weight_group:
                print("\t", weight)
            print("]")
        


#### **Preprocessing Data**

In [143]:
class JsonModelParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self.load_json_file()
        self.parse_model_data()

    def printDetails(self):
        print("\tINPUT SIZE:",self.input_size)
        print("\tLAYERS:", self.layers)
        print("\tINPUT:", self.input)
        print("\tINITIAL WEIGHTS:", self.initial_weights)
        print("\tTARGET:", self.target)
        print("\tLEARNING RATE:", self.learning_rate)
        print("\tBATCH SIZE:", self.batch_size)
        print("\tMAX ITERATION:", self.max_iteration)
        print("\tERROR THRESHOLD:", self.error_threshold)
    

    def load_json_file(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"The file {self.filepath} was not found")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {self.filepath}")
            return None

    def parse_model_data(self):
        if self.data:
            self.case = self.data.get('case', {})
            self.model = self.case.get('model', {})
            self.input_size = self.model.get('input_size')

            raw_layers = self.model.get('layers', [])   
            self.layers = [{'number_of_neurons': layer.get('number_of_neurons'),
                        'activation_function': layer.get('activation_function')}
                       for layer in raw_layers]
            
            self.input = self.case.get('input', [])
            self.initial_weights = self.case.get('initial_weights', [])
            self.target = self.case.get('target', [])
            self.parameters = self.case.get('learning_parameters', {})

            self.learning_rate = self.parameters.get('learning_rate')
            self.batch_size = self.parameters.get('batch_size')
            self.max_iteration = self.parameters.get('max_iteration')
            self.error_threshold = self.parameters.get('error_threshold')

            self.expect = self.data.get('expect', {})
            self.stopped_by = self.expect.get('stopped_by', '')
            self.final_weights = self.expect.get('final_weights', [])

    @staticmethod
    def save_json_file(data, filepath):
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
        except IOError:
            print(f"Could not save data to {filepath}")

#### **Processing Data**

In [144]:
def sum_squared_error(weights1, weights2):
    sse = 0
    for layer1, layer2 in zip(weights1, weights2):
        for neuron_weights1, neuron_weights2 in zip(layer1, layer2):
            for weight1, weight2 in zip(neuron_weights1, neuron_weights2):
                diff = weight1 - weight2
                sse += diff * diff
    return sse

In [145]:

# Read all files inside the test folders
import os

currDir = os.getcwd()
testDir = currDir.replace("src", "test")

testFiles = os.listdir(testDir)
testFilesArr = []

for filename in testFiles:
  if ("layer.json" in filename):
    testFilesArr.append(testDir+"\\"+filename)


# Use every file inside testFilesArr as input

for testFile in testFilesArr:
  print("FILENAME:", os.path.basename(testFile))
  architecture = JsonModelParser(testFile)
  architecture.printDetails()
  print("\n")
  model = ArtificialNeuralNetwork(architecture)
  model.train()

  error = sum_squared_error(model.weights, architecture.final_weights)

  print("\n")
  print("SSE (SUM SQUARED ERROR):", error)
  print("====================================================================")
  print("====================================================================")
  print("\n")

FILENAME: softmax_two_layer.json
	INPUT SIZE: 2
	LAYERS: [{'number_of_neurons': 4, 'activation_function': 'relu'}, {'number_of_neurons': 2, 'activation_function': 'softmax'}]
	INPUT: [[3.99, 2.96], [-0.71, 2.8], [-2.43, -0.2], [-1.9, 2.62], [-2.58, 1.43], [-3.43, -0.25], [1.15, -2.3], [4.28, 3.45]]
	INITIAL WEIGHTS: [[[0.1, -0.1, 0.1, -0.1], [-0.1, 0.1, -0.1, 0.1], [0.1, 0.1, -0.1, -0.1]], [[0.12, -0.1], [-0.12, 0.1], [0.12, -0.1], [-0.12, 0.1], [0.02, 0.0]]]
	TARGET: [[0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1]]
	LEARNING RATE: 0.1
	BATCH SIZE: 1
	MAX ITERATION: 200
	ERROR THRESHOLD: 0.01


[0.58685036 0.41314964]
[0, 1]
RES [-0.5868503570425038, 0.5868503570425039]
[0.50181578 0.49818422]
[1, 0]
RES [0.4981842152259619, -0.49818421522596196]
[0.51749679 0.48250321]
[0, 1]
RES [-0.5174967899863836, 0.5174967899863836]
[0.49493153 0.50506847]
[1, 0]
RES [0.5050684698947185, -0.5050684698947185]
[0.52421013 0.47578987]
[1, 0]
RES [0.47578986809920054, -0.4757898680992

C:\Users\Juan Christopher\AppData\Local\Temp\ipykernel_27696\1982708784.py:30: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (error_total < self.error_threshold):


[0.9759354 0.0240646]
[1, 0]
RES [0.024064601204405567, -0.024064601204405563]
[0.00651285 0.99348715]
[0, 1]
RES [-0.006512853169630493, 0.0065128531696305725]
[0.98347038 0.01652962]
[1, 0]
RES [0.016529617965057697, -0.016529617965057596]
[0.1000575 0.8999425]
[0, 1]
RES [-0.10005750194452824, 0.10005750194452834]
[0.09826991 0.90173009]
[0, 1]
RES [-0.09826991294817332, 0.09826991294817322]
[0.99889413 0.00110587]
[1, 0]
RES [0.0011058694790322399, -0.001105869479032116]
[0.0153856 0.9846144]
[0, 1]
RES [-0.015385603459659265, 0.015385603459659336]
[9.99855786e-01 1.44213676e-04]
[1, 0]
RES [0.00014421367645256478, -0.00014421367645255128]
[0.97669001 0.02330999]
[1, 0]
RES [0.023309990621552035, -0.02330999062155209]
[0.00617759 0.99382241]
[0, 1]
RES [-0.006177594164491541, 0.0061775941644915955]
[0.98435283 0.01564717]
[1, 0]
RES [0.015647170671848176, -0.015647170671848207]
[0.09721927 0.90278073]
[0, 1]
RES [-0.0972192741054522, 0.09721927410545228]
[0.09517999 0.90482001]
[0,

[0.98961794 0.01038206]
[1, 0]
RES [0.010382055064721674, -0.01038205506472169]
[0.00310236 0.99689764]
[0, 1]
RES [-0.0031023581171687013, 0.0031023581171686354]
[0.993924 0.006076]
[1, 0]
RES [0.0060760046191539185, -0.006076004619153951]
[0.04658839 0.95341161]
[0, 1]
RES [-0.046588392608856576, 0.0465883926088565]
[0.04617626 0.95382374]
[0, 1]
RES [-0.046176264878658974, 0.04617626487865889]
[9.99569745e-01 4.30255064e-04]
[1, 0]
RES [0.000430255063614271, -0.0004302550636143711]
[0.00776669 0.99223331]
[0, 1]
RES [-0.007766694390848081, 0.0077666943908482056]
[9.99967945e-01 3.20551201e-05]
[1, 0]
RES [3.205512010950873e-05, -3.205512010944499e-05]
[0.98962978 0.01037022]
[1, 0]
RES [0.010370220215884962, -0.010370220215885056]
[0.00296878 0.99703122]
[0, 1]
RES [-0.002968781173654253, 0.002968781173654178]
[0.99408257 0.00591743]
[1, 0]
RES [0.005917430137581503, -0.005917430137581523]
[0.04582377 0.95417623]
[0, 1]
RES [-0.04582377447096482, 0.04582377447096475]
[0.04542472 0.9